<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Spark/blob/master/Aula_3_Output_e_Persist%C3%AAncia_dos_Dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.0.1 with hadoop 2.7 and Java 8. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

     |████████████████████████████████| 204.2MB 71kB/s 
     |████████████████████████████████| 204kB 50.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=cde5d422ecd419959dcb25d050fae8da158eb976cbe46fe2b284d178806c3c47
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Run a local spark session to test your installation:

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
spark

# Reading a CSV from google drive

Utilizando o Google Colab, é possível importar os datasets diretamente do Google Drive, sem ter que realizar o upload manual dos mesmos para a instância colab manualmente

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
spark.read\
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("delimiter", ",") \
  .csv("/content/drive/My Drive/Spark_Treinamento/vgsales.csv") \
  .show()

+----+--------------------+--------+----+--------+------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|   Genre|   Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+--------+------------+--------+--------+--------+-----------+------------+
| 259|           Asteroids|    2600|1980| Shooter|       Atari|     4.0|    0.26|     0.0|       0.05|        4.31|
| 545|     Missile Command|    2600|1980| Shooter|       Atari|    2.56|    0.17|     0.0|       0.03|        2.76|
|1768|             Kaboom!|    2600|1980|    Misc|  Activision|    1.07|    0.07|     0.0|       0.01|        1.15|
|1971|            Defender|    2600|1980|    Misc|       Atari|    0.99|    0.05|     0.0|       0.01|        1.05|
|2671|              Boxing|    2600|1980|Fighting|  Activision|    0.72|    0.04|     0.0|       0.01|        0.77|
|4027|          Ice Hockey|    2600|1980|  Sports|  Activision|    0.46|

# Output e Persistência dos Dataframes
Nessa aula, estaremos cobrindo as mais populares formas de se realizar output e persistência nos Dataframes do Spark bem como os modos de inserção suportados pelo mesmo

### Carregando o dataset videogamesales
Dataset a ser utilizado: https://www.kaggle.com/gregorut/videogamesales/data

In [ ]:
data_dir = "vgsales.csv"

#### Lendo os dados

In [ ]:
df = spark.read.load(data_dir, format="csv", inferSchema="true", header="true")

In [ ]:
df.printSchema()

# Output / Ações
Output ou ações são o que delimitam um Job, que por sua vez, consiste em um conjunto de transformações em um Dataframe encadeadas e tendo sempre na ponta uma ação final. **Algumas ações são passíveis de serem realizadas de forma distribuída nos executores, mas outras, em decorrência de sua natureza, exige a centralização dos dados na entidade coordenadora chamada driver**. Basicamente:
1. Toda ação realizada para persistir o Dataframe **pode** ser realizada de forma distribuída direto pelos executores
2. Toda ação realizada para trazer Dataframes Spark para estruturas de dados em Python **exigirá a concentração dos dados no driver**

# Lazy evaluation
Em conjunto com o conceito das **Ações**, temos também o conceito adotado pelo Spark e outras ferramentas de Big Data: **Lazy Evaluation**. É um conceito que define que **todas as transformações encadeadas só serão executadas quando houver uma ação**. Por exemplo, veja o trecho abaixo:
```python
df2 = df.select("col1", "col2") \
    .filter(f.col("col2") < 10) \
    .groupBy("col1") \
    .agg(f.sum("col2"))
```

Podemos notar 3 transformações: (1) *select*, (2) *filter* e (3) *groupBy + agg*. Nesse momento, é possível notar que nenhuma das transformações até então foram executadas devido ao conceito da *Lazy Evaluation* (outras ferramentas de Big Data também utilizam essa metodologia), ou seja, as transformações só serão de fato executadas quando houver uma ação. Como ainda não executamos nenhuma, as computações ainda não ocorrem de imediato, conforme visto na [Spark UI](192.168.1.7:8080).

A computação somente ocorrerá ao executarmos os trechos abaixo:
```python
df2.show()
df2.collect()
```

Lazy evaluation é interessante porque permite ao Spark, escolher em tempo de execução o melhor plano a ser seguido, com as otimizações passíveis de serem aplicadas em cada caso, dado as transformações utilizadas pelo programador

### Output simples

##### Printando Dataframes

In [ ]:
df.show(5)

##### Convertendo todo o Dataframe Spark para estruturas em Python puro
Obs.: Cuidado!! Esses comandos podem resultar em falta de memoria no processo driver caso o Dataframe seja muito grande. É recomendado utilizá-lo apenas com Dataframes pequenos

In [ ]:
df.limit(10).collect() # converte o dataframe para uma lista em Python
df.limit(10).toPandas() # converte o dataframe Spark para um dataframe Pandas
df.limit(10).first() # retorna a primeira linha do Dataframe Spark como uma lista em Python

##### Contando o número de linhas

In [ ]:
df.count()

### Output no formato de persistência como uma tabela relacional
O Spark, além de ser uma engine de processamento paralelo, possui também um catálogo de dados onde é possível persistir informações de tabelas e utilizar a API do Spark SQL para consultá-las. Para interagir com esse catálogo de dados do Spark, temos duas formas:
- Criando tabelas temporárias: `createOrReplaceTempView()`
- Criando tabelas definitivas: `df.write.saveAsTable()`

In [ ]:
df.createOrReplaceTempView("temp_vgsales")
df.write.saveAsTable("stored_vgsales")

#### Visualizando as tabelas criadas

In [ ]:
spark.sql("SHOW TABLES").show()

### Output no formato de persistência em arquivos
Ao persistir dataframes em arquivos, o Spark oferece uma gama de possíveis arquivos de saída. Entretando, é necessário ficar atento a alguns pontos sobre os arquivos gerados.

1. Para todo output de arquivo gerado, será criado um diretório com o nome escolhido e dentro dele, haverá um ou mais arquivos `part-0000...` contendo os dados propriamente ditos, bem como um arquivo `_SUCCESS` (flag indicando a finalização do processo de persistência do dado)
2. É possível controlar a quantidade de partições do arquivo final com o comando `df.coalesce()` ou `df.repartition()`
3. Existem 2 modos de realizar o processo de persistência do dado `overwrite` e `append`

#### Persistência em arquivo CSV - 1 partição

In [ ]:
# .coalesce(1) - move os dados internamente para UMA partição apenas
# .mode("overwrite") - substitui o diretório inteiro caso seja executado novamente
# .option("header", "true") - incluir o cabeçalho no CSV (específica do formato CSV)

df.coalesce(1)\
    .write\
    .mode("overwrite")\
    .option("header", "true")\
    .csv("./output/vgsales_csv")

In [ ]:
!ls output/vgsales_csv

In [ ]:
!head -n 5 output/vgsales_csv/part-00000-9c3a12b7-372c-423a-ac8a-b79ff389d4f2-c000.csv

#### Persistência em arquivo CSV - 2 partições

In [ ]:
# .repartition(2) - salva o dataframe utilizando 2 partições

df.repartition(2)\
    .write\
    .mode("overwrite")\
    .option("header", "true")\
    .csv("./output/vgsales_csv")

In [ ]:
!ls output/vgsales_csv

#### Persistência em arquivo CSV - 2 partições e modo 'append'

In [ ]:
# .mode("append") - realiza a operação de append ao salvar no mesmo diretório

df.repartition(2)\
    .write\
    .mode("append")\
    .option("header", "true")\
    .csv("./output/vgsales_csv")

In [ ]:
!ls output/vgsales_csv

#### Persistência em arquivo CSV - 1 partição tradicional e uma partição pela coluna Platform

In [ ]:
df.coalesce(1)\
    .write\
    .partitionBy("Platform") \
    .mode("overwrite")\
    .option("header", "true")\
    .csv("./output/vgsales_csv")

Lendo o dataset particionado

In [ ]:
spark.read.load("output/vgsales_csv", format="csv", inferSchema="true", header="true") \
  .printSchema()

#### Persistência em arquivo JSON

In [ ]:
df.coalesce(1)\
    .write\
    .mode("overwrite")\
    .json("./output/vgsales_json")

In [ ]:
!ls output/vgsales_json

In [ ]:
!head -n 5 output/vgsales_json/part-00000-0f205f75-4b8b-4b0d-b6d0-4558bb531a5f-c000.json

#### Persistência em arquivo Parquet

In [ ]:
df.coalesce(1)\
    .write\
    .mode("overwrite")\
    .parquet("./output/vgsales_parquet")

In [ ]:
!ls output/vgsales_parquet

### Observações
Além de possuir uma gama de formatos de entrada e saída de dados, o Spark também consegue ler e salvar esses dados das mais variadas fontes. Podemos citar algumas tais quais:
- Brokers como Kafka, MQTT, AWS Kinesis, etc
- Bancos de dados relacionais: MySQL, SQL Server, etc
- Cloud storages como S3, ADLS, Blob Storage
- Hadoop Distributed File System (HDFS)
- Bancos de dados NoSQL: Redis, HBase, Cassandra, MongoDB, Elasticsearch, timeseries DB, etc

# Exercícios

1) Exporte o dataframe vgsales em formato JSON, com 4 partições em modo overwrite ativado no caminho ./output/vgsales_json:

In [ ]:
# df = ...

2) Exporte o dataframe vgsales em formato CSV, com 2 partições tradicionais, e uma partição por coluna no campo Year.

In [ ]:
#df = ...